In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 10, 3, padding=1)
        self.conv4 = nn.Conv2d(10, 10, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3)
        self.conv6 = nn.Conv2d(10, 10, 3)
        self.conv7 = nn.Conv2d(10, 120, 3)
        self.fc = nn.Linear(120, 10)

        #Dropout layers
        self.dropout= nn.Dropout(0.1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.dropout(x)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.dropout(x)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.dropout(x)
        x = F.relu(self.conv7(x))
        x = x.view(x.size(0), -1) #flatten tensor
        x = self.fc(x)
        x = self.dropout(x)

        return F.log_softmax(x,dim=1)

In [ ]:
def check_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params}")
    assert total_params < 20000, f"Model has more than 20,000 parameters. Actual: {total_params}"

# Check if Batch Normalization is used in the model
def check_batch_normalization(model):
    batch_norm_layers = [module for module in model.modules() if isinstance(module, nn.BatchNorm2d)]
    print(f"Batch Normalization layers: {len(batch_norm_layers)}")
    assert len(batch_norm_layers) > 0, "Batch Normalization is not used."

# Check if Dropout is used in the model
def check_dropout(model):
    dropout_layers = [module for module in model.modules() if isinstance(module, nn.Dropout)]
    print(f"Dropout layers: {len(dropout_layers)}")
    assert len(dropout_layers) > 0, "Dropout is not used."

# Check if Fully Connected layers or Global Average Pooling (GAP) is used
def check_fc_or_gap(model):
    fc_layers = [module for module in model.modules() if isinstance(module, nn.Linear)]
    print(f"Fully Connected layers: {len(fc_layers)}")
    assert len(fc_layers) > 0, "No Fully Connected layers found."

# Check if the model's accuracy is greater than 99.4%


# Load the MNIST test data
def load_data(batch_size=128):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, transform=transform),
                                              batch_size=batch_size, shuffle=False)
    return test_loader

# Perform checks
check_model_parameters(model)
check_batch_normalization(model)
check_dropout(model)
check_fc_or_gap(model)


print("All checks passed!")


In [48]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 20, 28, 28]           1,820
         MaxPool2d-3           [-1, 20, 14, 14]               0
           Dropout-4           [-1, 20, 14, 14]               0
            Conv2d-5           [-1, 10, 14, 14]           1,810
            Conv2d-6           [-1, 10, 14, 14]             910
         MaxPool2d-7             [-1, 10, 7, 7]               0
           Dropout-8             [-1, 10, 7, 7]               0
            Conv2d-9             [-1, 10, 5, 5]             910
           Conv2d-10             [-1, 10, 3, 3]             910
          Dropout-11             [-1, 10, 3, 3]               0
           Conv2d-12            [-1, 120, 1, 1]          10,920
           Linear-13                   [-1, 10]           1,210
          Dropout-14                   

In [49]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [50]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [53]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.025, momentum=0.9)
scheduler= StepLR(optimizer, step_size=9, gamma=0.15)


for epoch in range(1, 20):
    print(f"Epoch {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1, Learning Rate: 0.025


loss=0.1951637715101242 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 18.01it/s]



Test set: Average loss: 0.1226, Accuracy: 9594/10000 (96%)

Epoch 2, Learning Rate: 0.025


loss=0.22928406298160553 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.77it/s]



Test set: Average loss: 0.0630, Accuracy: 9805/10000 (98%)

Epoch 3, Learning Rate: 0.025


...


Test set: Average loss: 0.0222, Accuracy: 9939/10000 (99%)

Epoch 18, Learning Rate: 0.00375


loss=0.06618211418390274 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.96it/s]



Test set: Average loss: 0.0219, Accuracy: 9940/10000 (99%)

Epoch 19, Learning Rate: 0.0005625


loss=0.11757007241249084 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 18.01it/s]



Test set: Average loss: 0.0209, Accuracy: 9941/10000 (99%)

